In [28]:
!pip install pyqtgraph
!pip install PySide6
!pip install PyQt6
!pip install PyQt5

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 17.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.8/38.8 MB 31.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.3/124.3 kB 4.5 MB/s eta 0:00:00


In [3]:
import pyqtgraph as pg
from pyqtgraph.Qt import QtCore, QtGui
import numpy as np

# Show a picture .
# Left mouse click to draw scatter dots on the picture. Hold down the left mouse button and drag to move the screen.
# Hold down the right mouse button to select a scatter point and drag the scatter point to the release position.
# The current coordinates are displayed when the mouse moves.
# Mouse wheel zooms in or out, screen.

class MovableScatterPlotItem(pg.ScatterPlotItem):
    def __init__(self, *args, imageSizeXy, **kargs):
        super().__init__(*args, **kargs)
        self.target = pg.TargetItem()
        self.target.setParentItem(self)
        self.target.sigPositionChanged.connect(self.targetMoved)
        self.target.hide()
        self.selectedPoint = None
        self.coordinateLabel = pg.TextItem()
        self.coordinateLabel.setParentItem(self.target)
        self.coordinateLabel.setAnchor((0, 1))
        self.imageSizeXy = imageSizeXy

    def boundingRect(self):
        return QtCore.QRectF(0, 0, *self.imageSizeXy)

    def targetMoved(self, target):
        if self.target.isVisible() and self.selectedPoint is not None:
            self.data[["x", "y"]][self.selectedPoint.index()] = tuple(target.pos())
            self.updateSpots()
            self.invalidate()
            label = f"{tuple(map(lambda el: round(el, 2), target.pos()))}"
            self.coordinateLabel.setHtml("<div style='color: red; background: black;'>%s</div>" % label)

    def mouseClickEvent(self, ev):
        if ev.button() == QtCore.Qt.MouseButton.RightButton:
            points = self.pointsAt(ev.pos())
            if len(points):
                self.target.setPos(ev.pos())
                self.selectedPoint = points[-1]
                self.target.show()
                ev.accept()
        elif ev.button() == QtCore.Qt.MouseButton.LeftButton:
            if self.target.isVisible():
                self.target.hide()
            else:
                newData = np.r_[np.c_[self.getData()], np.atleast_2d(ev.pos())]
                self.setData(*newData.T)
            ev.accept()
        else:
            super().mouseClickEvent(ev)

pg.mkQApp()
plot = pg.PlotWidget()
imageItem = pg.ImageItem(np.random.randint(256, size=(512, 512)))
#plot.addItem(imageItem)
scatter = MovableScatterPlotItem(imageSizeXy=imageItem.image.shape[:2][::-1], pen="w", brush="r", size=12)
# Init 3 random points across the image
randomPoints = np.random.randint(512, size=(2, 3))
scatter.setData(*randomPoints)
plot.addItem(scatter)
plot.show()
pg.exec()

0

In [2]:
import sys
from PyQt5.QtWidgets import QApplication, QWidget
from PyQt5.QtGui import QPainter, QPen, QBrush
from PyQt5.QtCore import Qt, QPoint
from IPython import get_ipython
class VertexWidget(QWidget):
    def __init__(self):
        super().__init__()
        self.vertices = [QPoint(50, 50), QPoint(150, 50), QPoint(100, 150)]
        self.selected_vertex_index = None
        self.setWindowTitle("Vertex Mover")
        self.setGeometry(100, 100, 400, 300)
        self.show()
    def paintEvent(self, event):
        painter = QPainter(self)
        painter.setRenderHint(QPainter.Antialiasing)
        painter.setPen(QPen(Qt.black, 2))
        painter.setBrush(QBrush(Qt.blue, Qt.SolidPattern))
        for vertex in self.vertices:
            painter.drawEllipse(vertex, 10, 10)
    def mousePressEvent(self, event):
        for index, vertex in enumerate(self.vertices):
            if (event.pos() - vertex).manhattanLength() < 10:
                self.selected_vertex_index = index
                break
    def mouseMoveEvent(self, event):
        if self.selected_vertex_index is not None:
            self.vertices[self.selected_vertex_index] = event.pos()
            self.update()
    def mouseReleaseEvent(self, event):
        self.selected_vertex_index = None
def run_app():
    app = QApplication.instance()
    if app is None:
        app = QApplication(sys.argv)
    widget = VertexWidget()
    app.exec_()
# Use the '%matplotlib qt' magic to enable the Qt event loop integration
get_ipython().run_line_magic('matplotlib', 'qt')
# Run the app
run_app()